<img src="https://github.com/hernancontigiani/ceia_memorias_especializacion/raw/master/Figures/logoFIUBA.jpg" width="500" align="center">


# Procesamiento de lenguaje natural
## Custom embedddings con Gensim



### Objetivo
El objetivo es utilizar documentos / corpus para crear embeddings de palabras basado en ese contexto. Se utiliziarán los siguientes poemas de Ursula K. Le Guin como corpus:

-Leaves
-Six Quatrains
-To the train
-Bats
-Looking Back
-How It Seems To Me
-McCoy Creek: Cattle
-Kinship
-Theory of anging
-The way
-Hymn to time
-The maenads
-Read at the award dinner
-Anonyma
-Up in a cottonwood
-On the second hill
-Desire and fear
-Coast range highway, November
-The salt
-Intimation
-Every land
-A Cannon beach crow album
-Learning the name
-Crows

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import multiprocessing
from gensim.models import Word2Vec

### Datos
Utilizaremos como dataset canciones de bandas de habla inglés.

In [5]:
import os
import platform
os.listdir("./Data/")

['ursulakleguin.txt']

In [8]:
# Armar el dataset utilizando salto de línea para separar los versos
df = pd.read_csv('Data/ursulakleguin.txt', sep='/n', header=None, engine='python')
df.head()

,0
0,Years do odd things to identity.
1,What does it mean to say
2,I am that child in the photograph
3,at Kishamish in 1935?
4,Might as well say I am the shadow


In [9]:
print("Cantidad de documentos:", df.shape[0])

Cantidad de documentos: 368


### 1 - Preprocesamiento

In [12]:
from keras.preprocessing.text import text_to_word_sequence

sentence_tokens = []
# Recorrer todas las filas y transformar las oraciones
# en una secuencia de palabras (esto podría realizarse con NLTK o spaCy también)
for _, row in df[:None].iterrows():
    sentence_tokens.append(text_to_word_sequence(row[0]))

In [13]:
# Demos un vistazo
sentence_tokens[:2]

[['years', 'do', 'odd', 'things', 'to', 'identity'],
 ['what', 'does', 'it', 'mean', 'to', 'say']]

### 2 - Crear los vectores (word2vec)

In [14]:
from gensim.models.callbacks import CallbackAny2Vec
# Durante el entrenamiento gensim por defecto no informa el "loss" en cada época
# Sobracargamos el callback para poder tener esta información
class callback(CallbackAny2Vec):
    """
    Callback to print loss after each epoch
    """
    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        if self.epoch == 0:
            print('Loss after epoch {}: {}'.format(self.epoch, loss))
        else:
            print('Loss after epoch {}: {}'.format(self.epoch, loss- self.loss_previous_step))
        self.epoch += 1
        self.loss_previous_step = loss

In [70]:
# Crearmos el modelo generador de vectoeres
# En este caso utilizaremos la estructura modelo Skipgram
w2v_model = Word2Vec(min_count=2,    # frecuencia mínima de palabra para incluirla en el vocabulario
                     window=2,       # cant de palabras antes y desp de la predicha
                     vector_size=40,       # dimensionalidad de los vectores 
                     negative=20,    # cantidad de negative samples... 0 es no se usa
                     workers=1,      # si tienen más cores pueden cambiar este valor
                     sg=1)           # modelo 0:CBOW  1:skipgram

In [71]:
# Buildear el vocabularui con los tokens
w2v_model.build_vocab(sentence_tokens)

In [72]:
# Cantidad de filas/docs encontradas en el corpus
print("Cantidad de docs en el corpus:", w2v_model.corpus_count)

Cantidad de docs en el corpus: 368


In [73]:
# Cantidad de words encontradas en el corpus
print("Cantidad de words distintas en el corpus:", len(w2v_model.wv))

Cantidad de words distintas en el corpus: 274


### 3 - Entrenar el modelo generador

In [75]:
# Entrenamos el modelo generador de vectores
# Utilizamos nuestro callback
w2v_model.train(sentence_tokens,
                 total_examples=w2v_model.corpus_count,
                 epochs=200,
                 compute_loss = True,
                 callbacks=[callback()]
                 )

Loss after epoch 0: 5457.49609375
Loss after epoch 1: 5532.71875
Loss after epoch 2: 5418.09765625
Loss after epoch 3: 5653.671875
Loss after epoch 4: 5574.712890625
Loss after epoch 5: 5373.220703125
Loss after epoch 6: 5522.9296875
Loss after epoch 7: 5485.45703125
Loss after epoch 8: 5565.265625
Loss after epoch 9: 5785.91796875
Loss after epoch 10: 5478.046875
Loss after epoch 11: 5496.36328125
Loss after epoch 12: 5229.765625
Loss after epoch 13: 5567.1875
Loss after epoch 14: 5402.5546875
Loss after epoch 15: 5201.2890625
Loss after epoch 16: 5201.296875
Loss after epoch 17: 5283.09375
Loss after epoch 18: 5409.3671875
Loss after epoch 19: 5023.2109375
Loss after epoch 20: 5247.171875
Loss after epoch 21: 4910.84375
Loss after epoch 22: 4806.5234375
Loss after epoch 23: 4597.453125
Loss after epoch 24: 4774.125
Loss after epoch 25: 4976.296875
Loss after epoch 26: 4482.875
Loss after epoch 27: 4571.484375
Loss after epoch 28: 4460.296875
Loss after epoch 29: 4422.71875
Loss after

(173952, 406200)

### 4 - Ensayar

In [76]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["tree"], topn=10)

[('slowly', 0.6933777332305908),
 ('another', 0.6923360824584961),
 ('sit', 0.6434286832809448),
 ('hill', 0.615181565284729),
 ('autumn', 0.5839930772781372),
 ('shadows', 0.580493688583374),
 ('wild', 0.5779005885124207),
 ('at', 0.5693116188049316),
 ('where', 0.554722249507904),
 ('root', 0.5483406186103821)]

In [77]:
# Palabras que MENOS se relacionan con...:
w2v_model.wv.most_similar(negative=["wind"], topn=10)

[('hold', -0.027474133297801018),
 ('feel', -0.06878168880939484),
 ('soul', -0.0776933878660202),
 ('die', -0.0964905247092247),
 ('misty', -0.10203393548727036),
 ('dance', -0.11683733016252518),
 ('mind', -0.12056729197502136),
 ('black', -0.1266278326511383),
 ('cry', -0.1278424710035324),
 ('when', -0.1289939284324646)]

In [78]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["dark"], topn=10)

[('bird', 0.8252364993095398),
 ('going', 0.6510608196258545),
 ('light', 0.6440236568450928),
 ('sit', 0.6319989562034607),
 ('home', 0.6093012094497681),
 ('autumn', 0.5976971983909607),
 ('wind', 0.5916298627853394),
 ('rimrock', 0.5910196900367737),
 ('sand', 0.5899736285209656),
 ('grey', 0.5895016193389893)]

In [79]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["still"], topn=5)

[('home', 0.8101037740707397),
 ('happy', 0.8064088821411133),
 ('go', 0.7251759767532349),
 ('down', 0.6939199566841125),
 ('woman', 0.6580080986022949)]

In [80]:
# Ensayar con una palabra que no está en el corpus (en vocab):
w2v_model.wv.most_similar(negative=["night"])

[('in', 0.08286332339048386),
 ('that', 0.00042860390385612845),
 ('at', -0.022469859570264816),
 ('shadows', -0.06383362412452698),
 ('leaves', -0.06749285012483597),
 ('to', -0.0987805724143982),
 ('wings', -0.10029339045286179),
 ('salt', -0.12174148857593536),
 ('flying', -0.12252240628004074),
 ('flickering', -0.12752129137516022)]

### 5 - Visualizar agrupación de vectores

In [81]:
from sklearn.decomposition import IncrementalPCA    
from sklearn.manifold import TSNE                   
import numpy as np                                  

def reduce_dimensions(model):
    num_dimensions = 2  

    vectors = np.asarray(model.wv.vectors)
    labels = np.asarray(model.wv.index_to_key)  

    tsne = TSNE(n_components=num_dimensions, random_state=0)
    vectors = tsne.fit_transform(vectors)

    x_vals = [v[0] for v in vectors]
    y_vals = [v[1] for v in vectors]
    return x_vals, y_vals, labels

In [82]:
# Graficar los embedddings en 2D
import plotly.graph_objects as go
import plotly.express as px

x_vals, y_vals, labels = reduce_dimensions(w2v_model)

MAX_WORDS=200
fig = px.scatter(x=x_vals[:MAX_WORDS], y=y_vals[:MAX_WORDS], text=labels[:MAX_WORDS])
fig.show() # esto para plotly en colab

D:\FACULTAD\ESPECIALIZACIÓN\4. CUARTO BIMESTRE\NLP\env\lib\site-packages\sklearn\manifold\_t_sne.py:800: FutureWarning:

The default initialization in TSNE will change from 'random' to 'pca' in 1.2.

D:\FACULTAD\ESPECIALIZACIÓN\4. CUARTO BIMESTRE\NLP\env\lib\site-packages\sklearn\manifold\_t_sne.py:810: FutureWarning:

The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.

